This is a quick tutortial on how to use Gaia source IDs to extract orbital parameters from the Gaia non-single star (NSS) data table. This table contains non-single-star orbital models for sources compatible with an orbital two-body solution, making it super useful for studying binaries in our samples. To read more about the NSS table and look at all the columns it returns, click the following link: https://gaia.aip.de/metadata/gaiadr3/nss_two_body_orbit/

In [1]:
import astropy.units as u
import astropy.coordinates as coord
from astroquery.gaia import Gaia
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib as mpl
from matplotlib import colormaps
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
%matplotlib inline 

First you'll want to read in the sample where the source IDs exist, or if you would like to search just one, grab the DR3 source ID.

For this tutorial, we will read in the included table. This table is from Leiner+2022's work studying the possibility of sub-subgiants (SSG) being underluminous RS Canumm Venaticorum (RS CVn).

In [2]:
Leiner_Sample = pd.read_fwf("Leiner2022.dat.txt", header=None)
Leiner_columns = ["source_ID","VSX_ID","TIC_ID","ra","dec","dist","G","bp_rp","RUWE","chi^2","N_AL","P_VSX","P_TESS","Amp","N_sec","Av","Class"]
Leiner_Sample.columns = Leiner_columns
Leiner_Sample

,source_ID,VSX_ID,TIC_ID,ra,dec,dist,G,bp_rp,RUWE,chi^2,N_AL,P_VSX,P_TESS,Amp,N_sec,Av,Class
0,2448158953886445696,ZTF J000020.00-030630.5,138662652,0.08336,-3.10850,1489,14.84,1.28,1.12,407.47,324,6.13,-999.0,-999.0,0,0.11,RS
1,2874226134920416512,CSS_J000131.5+324913,83957002,0.38142,32.82031,1238,14.71,1.36,1.00,400.64,399,13.05,-999.0,-999.0,0,0.14,SSG
2,385028910256998528,ZTF J000139.60+442609.4,259193712,0.41504,44.43595,694,12.83,1.11,0.95,620.98,442,3.31,-999.0,-999.0,0,0.27,RS
3,2753192410330966912,ZTF J000146.98+093917.0,408505366,0.44579,9.65473,1942,15.09,1.51,1.03,370.28,351,22.03,-999.0,-999.0,0,0.40,SSG
4,2745167586059626240,ZTF J000216.58+055915.0,403022015,0.56911,5.98752,732,11.98,1.22,1.42,1924.19,548,6.46,-999.0,-999.0,0,0.12,cut
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1890,2878050614319122048,CSS_J235152.6+342725,67877343,57.96921,34.45697,1156,14.27,1.28,0.93,319.32,370,7.24,-999.0,-999.0,0,0.17,RS
1891,1926270708882502656,ZTF J235229.51+453334.9,177396080,58.12298,45.55972,1840,15.38,1.52,1.00,421.70,422,13.40,-999.0,-999.0,0,0.38,SSG
1892,2868410543043370880,ZTF J235240.40+310945.8,125280992,58.16835,31.16275,1836,15.07,1.28,1.01,401.40,389,0.91,-999.0,-999.0,0,0.30,RS
1893,1939867681994193280,ZTF J235609.70+485623.0,431733760,59.04044,48.93974,1541,14.78,1.42,1.05,759.88,689,3.89,-999.0,-999.0,0,0.46,RS


For fun, let's pick specifically the stars classified as RS and SSG

In [3]:
ssg_mask = Leiner_Sample['Class'].isin(['SSG', 'RS'])
ssg_only = Leiner_Sample[ssg_mask]
ssg_only

,source_ID,VSX_ID,TIC_ID,ra,dec,dist,G,bp_rp,RUWE,chi^2,N_AL,P_VSX,P_TESS,Amp,N_sec,Av,Class
0,2448158953886445696,ZTF J000020.00-030630.5,138662652,0.08336,-3.10850,1489,14.84,1.28,1.12,407.47,324,6.13,-999.00,-999.00,0,0.11,RS
1,2874226134920416512,CSS_J000131.5+324913,83957002,0.38142,32.82031,1238,14.71,1.36,1.00,400.64,399,13.05,-999.00,-999.00,0,0.14,SSG
2,385028910256998528,ZTF J000139.60+442609.4,259193712,0.41504,44.43595,694,12.83,1.11,0.95,620.98,442,3.31,-999.00,-999.00,0,0.27,RS
3,2753192410330966912,ZTF J000146.98+093917.0,408505366,0.44579,9.65473,1942,15.09,1.51,1.03,370.28,351,22.03,-999.00,-999.00,0,0.40,SSG
5,2881005998495586304,ZTF J000311.16+385513.2,194142000,0.79654,38.92036,1292,13.70,1.34,1.04,771.15,730,1.45,2.94,0.13,1,0.28,RS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889,2854458943077603840,ZTF J235059.77+265643.0,129642715,57.74908,26.94528,1637,13.44,1.29,1.01,283.51,281,1.58,-999.00,-999.00,0,0.12,RS
1890,2878050614319122048,CSS_J235152.6+342725,67877343,57.96921,34.45697,1156,14.27,1.28,0.93,319.32,370,7.24,-999.00,-999.00,0,0.17,RS
1891,1926270708882502656,ZTF J235229.51+453334.9,177396080,58.12298,45.55972,1840,15.38,1.52,1.00,421.70,422,13.40,-999.00,-999.00,0,0.38,SSG
1892,2868410543043370880,ZTF J235240.40+310945.8,125280992,58.16835,31.16275,1836,15.07,1.28,1.01,401.40,389,0.91,-999.00,-999.00,0,0.30,RS


To search up the stars in the Gaia NSS database, we require the source IDs. Let's pull them out and make sure we have what we need!

In [4]:
source_id_SSG = ssg_only['source_ID']
source_id_SSG

0       2448158953886445696
1       2874226134920416512
2        385028910256998528
3       2753192410330966912
5       2881005998495586304
               ...         
1889    2854458943077603840
1890    2878050614319122048
1891    1926270708882502656
1892    2868410543043370880
1893    1939867681994193280
Name: source_ID, Length: 1723, dtype: int64

To get an understanding of NSS has to offer, we can load in the NSS table and look at the columns, feel free to explore the column names and look at the Gaia DR3 NSS table in the link included above

In [5]:
gaiadr3_table = Gaia.load_table('gaiadr3.nss_two_body_orbit')
print(gaiadr3_table)

for column in gaiadr3_table.columns:
  print(column.name)

TAP Table name: gaiadr3.gaiadr3.nss_two_body_orbit
Description: This table contains non-single-star orbital models for sources compatible with an orbital two-body solution. This covers astrometric binaries, spectroscopic binaries, eclipsing binaries and certain combinations thereof. Several possible models are hosted within the same table and they are indicated by the field \texttt{nssSolutionType}. The description of this latter lists all possible solution types considered for this release. Only a selection of parameters hosted in this table are provided here, depending on the solution. The details of those is given in the description of field \texttt{bitIndex}, which can also be used to extract the relevant elements of the correlation vector \texttt{corrVec}. Details about the formalism used to derive the parameters in this table are given in the on-line documentation, see Chapter~\ref{chap:cu4nss}. Warning: as a source may receive independent astrometric, spectroscopic or photometri

Lookin good! Let's query Gaia NSS for those parameters!

In [6]:
Leiner_query = "SELECT * \
FROM gaiadr3.gaia_source AS dr3 \
JOIN gaiadr3.nss_two_body_orbit USING (source_id) \
WHERE (dr3.source_id in" + str(tuple(source_id_SSG))+") AND (dr3.phot_g_mean_mag < 16) AND (dr3.non_single_star >= 1))"

Leiner_job = Gaia.launch_job(Leiner_query)
Leiner_data = Leiner_job.get_results()

I'll explain what's going on:
The first line is "SELECT *". This line selects all the columns in the NSS datatable for you to look at and manipulate. You can select specific columns if you'd like to figure that on your own!

The second and third line are "FROM gaiadr3.gaia_source AS dr3\ 
JOIN gaiadr3.nss_two_body_orbit USING (source_id) \". This states that from the Gaia source datatable (calling it dr3), and joining the datatable with the NSS table, we can use the information from the gaia_source table to extract orbital parameter data from NSS using prescribed source IDs.

The final line is "WHERE (dr3.source_id in" + str(tuple(source_id_SSG))+") AND (dr3.phot_g_mean_mag < 16) AND (dr3.non_single_star >= 1))". Essentially, we are taking out list of Gaia Source IDs, passing them through gaia_source, and outputting a datatable of all the columns we requested.

In [7]:
Leiner_data

solution_id,designation,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pm,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,nu_eff_used_in_astrometry,pseudocolour,pseudocolour_error,ra_pseudocolour_corr,dec_pseudocolour_corr,parallax_pseudocolour_corr,pmra_pseudocolour_corr,pmdec_pseudocolour_corr,astrometric_matched_transits,visibility_periods_used,astrometric_sigma5d_max,matched_transits,new_matched_transits,matched_transits_removed,ipd_gof_harmonic_amplitude,ipd_gof_harmonic_phase,ipd_frac_multi_peak,ipd_frac_odd_win,ruwe,scan_direction_strength_k1,scan_direction_strength_k2,scan_direction_strength_k3,scan_direction_strength_k4,scan_direction_mean_k1,scan_direction_mean_k2,scan_direction_mean_k3,scan_direction_mean_k4,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,phot_bp_n_contaminated_transits,phot_bp_n_blended_transits,phot_rp_n_contaminated_transits,phot_rp_n_blended_transits,phot_proc_mode,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error,rv_method_used,rv_nb_transits,rv_nb_deblended_transits,rv_visibility_periods_used,rv_expected_sig_to_noise,rv_renormalised_gof,rv_chisq_pvalue,rv_time_duration,rv_amplitude_robust,rv_template_teff,rv_template_logg,rv_template_fe_h,rv_atm_param_origin,vbroad,vbroad_error,vbroad_nb_transits,grvs_mag,grvs_mag_error,grvs_mag_nb_transits,rvs_spec_sig_to_noise,phot_variable_flag,l,b,ecl_lon,ecl_lat,in_qso_candidates,in_galaxy_candidates,non_single_star,has_xp_continuous,has_xp_sampled,has_rvs,has_epoch_photometry,has_epoch_rv,has_mcmc_gspphot,has_mcmc_msc,in_andromeda_survey,classprob_dsc_combmod_quasar,classprob_dsc_combmod_galaxy,classprob_dsc_combmod_star,teff_gspphot,teff_gspphot_lower,teff_gspphot_upper,logg_gspphot,logg_gspphot_lower,logg_gspphot_upper,mh_gspphot,mh_gspphot_lower,mh_gspphot_upper,distance_gspphot,distance_gspphot_lower,distance_gspphot_upper,azero_gspphot,azero_gspphot_lower,azero_gspphot_upper,ag_gspphot,ag_gspphot_lower,ag_gspphot_upper,ebpminrp_gspphot,ebpminrp_gspphot_lower,ebpminrp_gspphot_upper,libname_gspphot,solution_id_2,nss_solution_type,ra_2,ra_error_2,dec_2,dec_error_2,parallax_2,parallax_error_2,pmra_2,pmra_error_2,pmdec_2,pmdec_error_2,a_thiele_innes,a_thiele_innes_error,b_thiele_innes,b_thiele_innes_error,f_thiele_innes,f_thiele_innes_error,g_thiele_innes,g_thiele_innes_error,c_thiele_innes,c_thiele_innes_error,h_thiele_innes,h_thiele_innes_error,period,period_error,t_periastron,t_periastron_error,eccentricity,eccentricity_error,center_of_mass_velocity,center_of_mass_velocity_error,semi_amplitude_primary,semi_amplitude_primary_error,semi_amplitude_secondary,semi_amplitude_secondary_error,mass_ratio,mass_ratio_error,fill_factor_primary,fill_factor_primary_error,fill_factor_secondary,fill_factor_secondary_error,inclination,inclination_error,arg_periastron,arg_periastron_error,temperature_ratio,temperature_ratio_error,temperature_ratio_definition,astrometric_n_obs_al_2,astrometric_n_good_obs_al_2,rv_n_obs_primary,rv_n_good_obs_primary,rv_n_obs_secondary,rv_n_good_obs_secondary,phot_g_n_obs_2,phot_g_n_good_obs,bit_index,corr_vec,obj_func,goodness_of_fit,efficiency,significance,flags,conf_spectro_period,r_pole_sum,r_l1_point_sum,r_spher_sum,ecl_time_primary,ecl_time_secondary,ecl_dur_primary,ecl_dur_secondary,g_luminosity_ratio,input_period_error

In [8]:
Leiner_data_csv = Leiner_data.to_pandas()
Leiner_data_csv

,solution_id,designation,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,...,r_spher_sum,ecl_time_primary,ecl_time_secondary,ecl_dur_primary,ecl_dur_secondary,g_luminosity_ratio,input_period_error,g_rank,astrometric_jitter,source_id
0,1636148068921376768,Gaia DR3 293812978938542592,1482780366,2016.0,20.413638,0.016111,25.296445,0.015182,1.359922,0.022211,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,293812978938542592
1,1636148068921376768,Gaia DR3 381661101844556928,197009700,2016.0,9.138361,0.014246,43.098450,0.012039,3.116524,0.016468,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,381661101844556928
2,1636148068921376768,Gaia DR3 387451130075951360,124098262,2016.0,11.391138,0.010997,42.697026,0.009080,2.161020,0.013379,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,387451130075951360
3,1636148068921376768,Gaia DR3 389680699138908800,1233244080,2016.0,11.325330,0.011517,46.150773,0.009022,0.913327,0.013410,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,389680699138908800
4,1636148068921376768,Gaia DR3 498912918751087104,1182522071,2016.0,78.788431,0.009185,72.303342,0.010485,1.141031,0.014725,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,498912918751087104
5,1636148068921376768,Gaia DR3 584183485754518016,1786501113,2016.0,135.873014,0.022574,7.593801,0.014629,0.670721,0.026354,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,584183485754518016
6,1636148068921376768,Gaia DR3 655828900372514688,1611541273,2016.0,123.804004,0.015442,16.744855,0.011238,0.788495,0.018842,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,655828900372514688
7,1636148068921376768,Gaia DR3 863106873934915200,1587778851,2016.0,176.099607,0.010405,62.969380,0.010406,1.470703,0.013689,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,863106873934915200
8,1636148068921376768,Gaia DR3 936457665445032576,401777909,2016.0,121.038024,0.018100,53.278439,0.014174,2.290432,0.028443,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,936457665445032576
9,1636148068921376768,Gaia DR3 1081565094046074624,427408374,2016.0,120.648796,0.018943,57.273363,0.020429,10.041267,0.029502,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1081565094046074624


You now have all the stars in your sample which have non-singular solutions! You now have the power to use these solutions and compare it to your data!